In [4]:
import requests
import json
from collections import defaultdict
from pprint import pprint

NGRAM_API_URI = "https://{0}.linggle.com/query/"
EXP_API_URI = "https://{0}.linggle.com/example/"

In [5]:
class Linggle:
    def __init__(self, ver='www'):
        self.ver = ver

    def __getitem__(self, query):
        return self.search(query)

    def search(self, query): #回傳query的所有結果
        query = query.replace('/', '@')
        req = requests.get(NGRAM_API_URI.format(self.ver) + query)
        results = req.json()
        return results.get('ngrams', [])

    def get_example(self, ngram_str):
        res = requests.post(EXP_API_URI.format(self.ver), json={'ngram': ngram_str})
        if res.status_code == 200:
            result = res.json()
            return result.get("examples", [])
        return []

In [6]:
def readFile(path):
    with open(path,encoding='utf-8') as f:
        return json.load(f)


In [27]:
def getAllCombination(head):
    return ling.search(head + ' _ _')

In [45]:
def getTag(w1,w2):
    words = w1 + " " + w2

    tags = biposgram[words] if words in biposgram else uniposgram[words] if w2 in uniposgram else {}
    totalCount = sum(list(tags.values()))
    
    probs = {key : value/totalCount for key,value in tags.items()}
    tag = max(probs, key=lambda k: probs[k])
    tag = change[tag] if tag not in boringType and tag in change else ''
   
    return tag

In [11]:
def getExamples(pattern):
    for query in result[pattern]:
        examples = ling.get_example(query)
        if len(examples)!= 0: 
            return examples[0]
                        
    return ''

In [12]:
bigram = readFile(r'D:\Tammy\University\graduate\107-spring\NLP\lab_4\bi.json')
unigram = readFile(r'D:\Tammy\University\graduate\107-spring\NLP\lab_4\un.json')
biposgram = readFile(r'D:\Tammy\University\graduate\107-spring\NLP\lab_4\pos.ngram.json')
uniposgram = readFile(r'D:\Tammy\University\graduate\107-spring\NLP\lab_4\uni.pos.json')

In [13]:
change = readFile(r'D:\Tammy\University\graduate\107-spring\NLP\lab_4\change.json')

In [46]:
ling = Linggle()

head = 'happy'
comb = getAllCombination(head)

result = defaultdict(lambda:defaultdict(lambda:0))
boringType = {'CD','DT','EX','FW','LS','MD','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','SYM','UH','WDT','WP','WP$','WRB'}

for item in comb:
    words = item[0]
    splits = words.split() 
    w1 = splits[0]
    w2 = splits[1]
    w3 = splits[2]  
    count = item[1]
    
    tag2 = getTag(w1,w2)
    tag3 = getTag(w2,w3)
    
    if tag2!=''and tag3!='': 
        result[w1 + ' ' + tag2 + ' ' + tag3][w1 + ' ' + w2 + ' ' + w3] = count

pprint(result)

defaultdict(<function <lambda> at 0x000002094CE28D08>,
            {'happy adj. n.': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000002094CE286A8>,
                                          {'happy new year': 469100}),
             'happy conj. adj.': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000002094CE28950>,
                                             {'happy and healthy': 93227}),
             'happy n. n.': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000002094CE288C8>,
                                        {'happy New Year': 64717}),
             'happy n. to': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000002094CE28D90>,
                                        {'happy birthday to': 86999}),
             'happy to v.': defaultdict(<function <lambda>.<locals>.<lambda> at 0x00000208A6CACAE8>,
                                        {'happy to accept': 116389,
                                         'happy to announce': 167461,
         

In [47]:
patterns = {pattern : sum(list(queries.values())) for pattern,queries in result.items()} 
        
values = list(patterns.values())
keys = list(patterns.keys())
topPatterns = [keys[values.index(value)] for value in sorted(values,reverse = True)[:3]]

pprint(topPatterns)

['happy to v.', 'happy adj. n.', 'happy conj. adj.']


In [48]:
for pattern in topPatterns:
    pprint('pattern: ' + pattern + ' example: '+ getExamples(pattern))

'pattern: happy to v. example: # HOMER # Happy to be alive ...'
('pattern: happy adj. n. example: Merry Christmas , Happy Hanukkah , Happy '
 "Solstice , Happy Socrates ' birthday and Happy New Year !")
('pattern: happy conj. adj. example: Based on twenty-four years of '
 'professional experience , @@ continual review of articles and texts on music '
 'education , I can assure you that being a " happy and healthy " music '
 'educator is predicated on three fundamental principles -- organization , '
 'organization , organization !')
